Cleaned Version of a Decoder-only (GPT style) transformer architecture.

# Imports

In [2]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors
from tokenizers.normalizers import NFKC, Sequence
import os
from typing import List, Optional, Union
from tqdm.notebook import tqdm
from tqdm import tqdm
import random

import transformers
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, GPT2TokenizerFast
from transformers import PreTrainedTokenizerFast
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, DataCollatorForLanguageModeling
from datasets import Dataset, DatasetDict,load_dataset #, load_metric
from transformers.tokenization_utils_base import AddedToken

# import tensorflow as tf

from sklearn.model_selection import train_test_split

import torch


import ast # Because the tokenized_text column data is stored as a string instead of a list...
import pandas as pd



This makes use of the Hugging Face transformer.trainer API so it uses a PyTorch backend

In [3]:
# Check pytorch GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'You are usisng {device} on a {torch.cuda.get_device_name()}.')

You are usisng cuda on a NVIDIA GeForce GTX 1070.


# Grab the dataset

1) Get the dataset
2) Fix the tokenize_text column from a string to a list of strings
3) Pull out all of the words into a mega-list

**SOMTHING IS WRONG WITH THE DATASET.  Likely an issue with the preprocessing steps...**

See:

Item 19152 seems like it is too many cards put together... I need to figure out what is going on.

'Crash Landing — Search your library for a basic land card , reveal it , put it into your hand , then shuffle . \\n Goblin Camp — Create a Treasure token . \\n Emerald Grove — Create a 2/2 white Knight creature token . \\n Auntie\'s Teahouse — Scry 3 . \\n Defiled Temple — You may sacrifice a permanent . If you do , draw a card . \\n Mountain Pass — You may put a land card from your hand onto the battlefield . \\n Ebonlake Grotto — Create two 1/1 blue Faerie Dragon creature tokens with flying . \\n Grymforge — For each opponent , goad up to one target creature that player controls . \\n Githyanki Crèche — Distribute three +1/+1 counters among up to three target creatures you control . \\n Last Light Inn — Draw two cards . \\n Reithwin Tollhouse — Roll 2d4 and create that many Treasure tokens . \\n Moonrise Towers — Instant and sorcery spells you cast this turn cost 3 less to cast . \\n Gauntlet of Shar — Each opponent loses 5 life . \\n Balthazar\'s Lab — Return up to two target creature cards from your graveyard to your hand . \\n Circus of the Last Days — Create a token that\'s a copy of one of your commanders , except it\'s not legendary . \\n Undercity Ruins — Create three 4/1 black Skeleton creature tokens with menace . \\n Steel Watch Foundry — You get an emblem with "Creatures you control get +2/+2 and have trample . " \\n Ansur\'s Sanctum — Reveal the top four cards of your library and put them into your hand . Each opponent loses life equal to those cards\' total mana value . \\n Temple of Bhaal — Creatures your opponents control get -5/-5 until end of turn .'


In [4]:
# Read the processed card data from wherever you have it.

# df = pd.read_csv('../data/processed/mtg_carddata_processed.csv')
df = pd.read_csv('../data/processed/mtg_carddata_processed_2_23_25.csv')

In [5]:
df[:7]

,name,mana_cost,type_line,oracle_text,power,toughness,colors,keywords,mtgo_id,loyalty,defense,processed_oracle_text,tokenized_text,tfidf_vector
0,"Nissa, Worldsoul Speaker",{3}{G},Legendary Creature — Elf Druid,"Landfall — Whenever a land you control enters,...",3,3,['G'],['Landfall'],NaN,NaN,NaN,Landfall — Whenever a land you control enters ...,"['Landfall', 'Whenever', 'a', 'land', 'you', '...",[0. 0. 0. ... 0. 0. 0.]
1,Static Orb,{3},Artifact,"As long as <name> is untapped, players can't u...",NaN,NaN,[],[],15870.0,NaN,NaN,"As long as Static Orb is untapped , players ca...","['As', 'long', 'as', '<name>', 'is', 'untapped...",[0. 0. 0. ... 0. 0. 0.]
2,Sensory Deprivation,{U},Enchantment — Aura,Enchant creature\r\nEnchanted creature gets -3...,NaN,NaN,['U'],['Enchant'],49283.0,NaN,NaN,Enchant creature \n Enchanted creature gets -3...,"['Enchant', 'creature', '\\n', 'Enchanted', 'c...",[0. 0. 0. ... 0. 0. 0.]
3,Road of Return,{G}{G},Sorcery,Choose one —\r\n• Return target permanent card...,NaN,NaN,['G'],['Entwine'],77122.0,NaN,NaN,Choose one — \n • Return target permanent card...,"['Choose', 'one', '\\n', 'Return', 'target', '...",[0. 0. 0. ... 0. 0. 0.]
4,Storm Crow,{1}{U},Creature — Bird,Flying (This creature can't be blocked except ...,1,2,['U'],['Flying'],22609.0,NaN,NaN,Flying (This creature can't be blocked except ...,"['Flying', 'This', 'creature', ""can't"", 'be', ...",[0. 0. 0. ... 0. 0. 0.]
5,Snarlfang Vermin,{B},Creature — Rat,Whenever <name> deals combat damage to a creat...,2,1,['B'],[],NaN,NaN,NaN,Whenever Snarlfang Vermin deals combat damage ...,"['Whenever', '<name>', 'deals', 'combat', 'dam...",[0. 0. 0. ... 0. 0. 0.]
6,Walking Sponge,{1}{U},Creature — Sponge,{T}: Target creature loses your choice of flyi...,1,1,['U'],[],12637.0,NaN,NaN,{T} : Target creature loses your choice of fly...,"['{T}', ':', 'Target', 'creature', 'loses', 'y...",[0. 0. 0. ... 0. 0. 0.]


The tokenized_text here isn't really tokenized... it is just the text split into a list of words.

In [6]:

df['tokenized_text'][0]

"['Landfall', 'Whenever', 'a', 'land', 'you', 'control', 'enters', ',', 'you', 'get', '{E}', '{E}', 'two', 'energy', 'counters', '.', '\\\\n', 'You', 'may', 'pay', 'eight', '{E}', 'rather', 'than', 'pay', 'the', 'mana', 'cost', 'for', 'permanent', 'spells', 'you', 'cast', '.']"

In [ ]:
# Apply ast.literal_eval to each row in the tokenized_text column
df['tokenized_text'] = df['tokenized_text'].apply(ast.literal_eval)

# Display first few rows to verify
df['tokenized_text'].head(10)

In [ ]:
df['tokenized_text'][0][0]

In [ ]:
# Combine all tokens into one large list
all_tokens = []
for tokens in df['tokenized_text']:
    all_tokens.extend(tokens)

# Alternative one-liner using list comprehension
# all_tokens = [token for tokens in df['tokenized_text'] for token in tokens]

# Display the first 20 tokens to verify
print(f"Total tokens: {len(all_tokens)}")
print("First 20 tokens:", all_tokens[:20])

In [ ]:
corpus = df['processed_oracle_text']
print(corpus[0:3])

In [ ]:
# Convert corpus to list of strings if it's not already
corpus_list = corpus.tolist() if hasattr(corpus, 'tolist') else list(corpus)

In [ ]:
type(corpus)

In [ ]:
type(corpus_list[0])

In [ ]:
# Convert pandas Series to list of strings
corpus_list = corpus.values.tolist()

# Ensure all elements are strings
corpus_list = [str(text) for text in corpus_list]

In [ ]:
# Corrected the above code to ensure all elements are strings
corpus_list[0]

In [ ]:
# Check the first few elements to verify
print(type(corpus_list))  # Should show: <class 'list'>
print(type(corpus_list[0]))  # Should show: <class 'str'>

# Split into train/val/test datasets

Do we need to keep other information with the corpus_list as we split it up?  Names, color, etc.?

If we're just generating text, I think that answer is no.  If we want more, then the answer is likely yes.

In [ ]:
# Split data into train/validation/test (80/10/10) sets
train_list, temp = train_test_split(corpus_list, test_size=0.2, random_state=42) # Set 80% for training
val, test = train_test_split(temp, test_size=0.5, random_state=42) # Set 10% for validation and 10% for testing

In [ ]:
# Turn them into dicts so I can use them with the Hugging Face Dataset class
train = [{"sentence": text} for text in train_list]  # Wrap each sentence in a dict
val = [{"sentence": text} for text in val]
test = [{"sentence": text} for text in test]

# Create the Dataset Dictionary for future mapping
data = DatasetDict({
    'train': Dataset.from_list(train),
    'validation': Dataset.from_list(val),
    'test': Dataset.from_list(test)
    })

data

In [ ]:
# Check the items
data['train'][:5]

In [ ]:
data['validation']['sentence'][0]

# Train the Tokenizer on Training Data

Keep the validation and test sets out of the tokenizer to prevent data leakage.

In [ ]:
# Create a directory for tokenizer files if it doesn't exist
models_dir = "../models"
os.makedirs(models_dir, exist_ok=True)

## Tokenizer Functions

1) `create_gpt2_tokenizer`

2) `train_tokenizer_from_texts`

3) `convert_to_transformers_tokenizer` -> This is important to allow `trainer.train` to work

4) `use_transformers_tokenizer`

5) `load_transformers_tokenizer` -> Used if you have one to load, else just carry from `tokenizer=convert_to_transformers_tokenizer`

6) `demonstrate_workflow2` -> make sure this whole thing even works...

In [ ]:
def create_gpt2_tokenizer(vocab_size=8192, min_frequency=2):
    """
    Create a GPT2-style BPE tokenizer from scratch using Hugging Face tokenizers library.

    Args:
        vocab_size: The size of the vocabulary to learn
        min_frequency: Minimum frequency for a token to be considered in the BPE algorithm

    Returns:
        A tokenizer object ready for training
    """
    # Initialize a ByteLevelBPE model-based tokenizer
    tokenizer = Tokenizer(models.BPE())

    # Add byte-level pre-tokenizer
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

    # Add decoder to properly decode byte-level tokens
    tokenizer.decoder = decoders.ByteLevel()

    # Set up normalizers - GPT-2 doesn't do much normalization
    tokenizer.normalizer = Sequence([NFKC()])

    # Return the tokenizer (to be trained later)
    return tokenizer


def train_tokenizer_from_texts(
    tokenizer: Tokenizer,
    texts: List[str],
    vocab_size: int = 8192,
    min_frequency: int = 2,
    batch_size: int = 512,
    output_dir: str = "tokenizer"
):
    """
    Train the tokenizer on a list of texts using batching

    Args:
        tokenizer: The tokenizer object to train
        texts: List of strings to use for training
        vocab_size: Maximum vocabulary size
        min_frequency: Minimum frequency for a token
        batch_size: Number of texts to process in each batch
        output_dir: Directory to save the trained tokenizer

    Returns:
        The trained tokenizer
    """
    if not texts:
        raise ValueError("No texts provided for training")

    # Configure the BPE trainer
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        min_frequency=min_frequency,
        special_tokens=["<|endoftext|>", "<|pad|>"],
        show_progress=True,
    )

    # Create temporary files for batched training
    temp_dir = os.path.join(output_dir, "temp")
    os.makedirs(temp_dir, exist_ok=True)

    # Split texts into batches
    batches = [texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
    batch_files = []

    print(f"Preparing {len(batches)} batches for training...")

    # Write batches to temporary files
    for i, batch in enumerate(tqdm(batches)):
        batch_file = os.path.join(temp_dir, f"batch_{i}.txt")
        with open(batch_file, "w", encoding="utf-8") as f:
            f.write("\n<|endoftext|>\n".join(batch))
            f.write("\n<|endoftext|>\n")  # Add final separator
        batch_files.append(batch_file)

    # Train the tokenizer
    print("Training tokenizer...")
    tokenizer.train(batch_files, trainer)

    # Add post-processor to handle special tokens
    tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

    # Enable padding with the PAD token
    pad_id = tokenizer.token_to_id("<|pad|>")
    if pad_id is not None:
        tokenizer.enable_padding(pad_id=pad_id, pad_token="<|pad|>")
    else:
        print("Warning: <|pad|> token not found in vocabulary. Padding won't work correctly.")

    # Save the trained tokenizer
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    tokenizer.save(f"{output_dir}/gpt2_tokenizer.json")

    # Clean up temporary files
    for file in batch_files:
        os.remove(file)
    os.rmdir(temp_dir)

    print(f"Tokenizer trained and saved to {output_dir}/gpt2_tokenizer.json")
    return tokenizer


def convert_to_transformers_tokenizer(
    tokenizer_path: str,
    output_dir: str = None,
    model_max_length: int = 1024
):
    """
    Convert a trained tokenizers.Tokenizer to a transformers compatible tokenizer

    Args:
        tokenizer_path: Path to the saved tokenizer JSON file
        output_dir: Directory to save the transformers tokenizer
        model_max_length: Maximum sequence length for the model

    Returns:
        A transformers-compatible tokenizer
    """
    # Load the tokenizer
    if isinstance(tokenizer_path, Tokenizer):
        # If a tokenizer object is passed directly
        raw_tokenizer = tokenizer_path
    else:
        # Load from file
        raw_tokenizer = Tokenizer.from_file(tokenizer_path)

    # Define special tokens
    eos_token = "<|endoftext|>"  # GPT-2 uses EOS as both BOS and EOS
    pad_token = "<|pad|>"

    # Create the transformers wrapper
    transformers_tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=raw_tokenizer,
        bos_token=eos_token,  # GPT-2 uses EOS as BOS too
        eos_token=eos_token,
        pad_token=pad_token,
        unk_token=eos_token,  # GPT-2 doesn't really have UNK, defaults to EOS
        model_max_length=model_max_length
    )

    # Set additional GPT-2 specific attributes
    transformers_tokenizer.add_special_tokens({
        "eos_token": eos_token,
        "bos_token": eos_token,
        "pad_token": pad_token,
    })

    # Save the transformers tokenizer if an output directory is provided
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        transformers_tokenizer.save_pretrained(output_dir)
        print(f"Transformers tokenizer saved to {output_dir}")

    return transformers_tokenizer


def use_transformers_tokenizer(tokenizer, text, padding=True, truncation=True, max_length=None, return_tensors=None):
    """
    Demonstrate how to use the transformers tokenizer

    Args:
        tokenizer: The transformers tokenizer
        text: Text to tokenize or list of texts
        padding: Whether to pad the sequences (True, False, 'max_length', 'longest')
        truncation: Whether to truncate sequences (True, False)
        max_length: Maximum length for padding/truncation (defaults to model_max_length if None)
        return_tensors: Return format for tensors ('pt' for PyTorch, 'tf' for TensorFlow, None for lists)

    Returns:
        Encoding object(s) with tokens, ids, etc.
    """
    # Configure parameters
    if max_length is None:
        max_length = tokenizer.model_max_length

    # Encode the text(s)
    encoded = tokenizer(
        text,
        padding=padding,
        truncation=truncation,
        max_length=max_length,
        return_tensors=return_tensors
    )

    # Display results for single text if it's a string
    if isinstance(text, str):
        print(f"Input text: {text}")
        print(f"Token IDs: {encoded['input_ids'].tolist()[0] if return_tensors else encoded['input_ids']}")
        tokens = tokenizer.convert_ids_to_tokens(
            encoded['input_ids'].tolist()[0] if return_tensors else encoded['input_ids']
        )
        print(f"Tokens: {tokens}")

        if padding:
            print(f"Attention mask: {encoded['attention_mask'].tolist()[0] if return_tensors else encoded['attention_mask']}")

        # Decoding demonstration
        decoded = tokenizer.decode(
            encoded['input_ids'].tolist()[0] if return_tensors else encoded['input_ids'],
            skip_special_tokens=True
        )
        print(f"Decoded text: {decoded}")

    return encoded


def load_transformers_tokenizer(path):
    """
    Load a previously saved transformers tokenizer

    Args:
        path: Path to the saved transformers tokenizer directory

    Returns:
        The loaded transformers tokenizer
    """
    return GPT2TokenizerFast.from_pretrained(path)


def demonstrate_workflow2():
    """
    Full demonstration of creating, training, and using a GPT-2 style tokenizer
    that's compatible with the transformers library
    """
    # Create a new tokenizer
    print("Creating tokenizer...")
    tokenizer = create_gpt2_tokenizer()

    # Generate some sample texts for demonstration
    print("Generating sample texts...")
    sample_texts = [
        "This is an example of text that could be used to train a tokenizer.",
        "It should include diverse vocabulary, punctuation (like commas, periods, question marks?).",
        "Multiple paragraphs are good to include.",
        "Numbers like 42, 3.14159, and 2023 should be represented.",
        "Code snippets might be important if your model will process code:\ndef hello_world():\n    print('Hello, world!')",
    ]

    # Add some more generated texts for better training
    for i in range(50):
        words = ["The", "quick", "brown", "fox", "jumps", "over", "lazy", "dog",
                "Hello", "world", "Python", "programming", "is", "fun", "GPT",
                "natural", "language", "processing", "models", "work", "well"]
        length = random.randint(5, 20)
        text = " ".join(random.choices(words, k=length)) + "."
        sample_texts.append(text)

    # Train the tokenizer on the sample texts
    print(f"Training tokenizer on {len(sample_texts)} texts...")
    raw_tokenizer = train_tokenizer_from_texts(
        tokenizer,
        sample_texts,
        vocab_size=1000,  # Smaller vocab for demo
        min_frequency=1,
        batch_size=20,
        output_dir="tokenizer_demo"
    )

    # Convert to transformers tokenizer
    print("\nConverting to transformers tokenizer...")
    transformers_tokenizer = convert_to_transformers_tokenizer(
        "tokenizer_demo/gpt2_tokenizer.json",
        output_dir="transformers_tokenizer_demo",
        model_max_length=512
    )

    # Test the transformers tokenizer
    print("\nTesting transformers tokenizer...")
    test_texts = [
        "This is a short text.",
        "This is a slightly longer text with more content.",
        "Let's see how padding works on texts of different lengths."
    ]

    # Encode with transformers tokenizer (return PyTorch tensors)
    print("\nEncoding with PyTorch tensors:")
    encodings = use_transformers_tokenizer(
        transformers_tokenizer,
        test_texts,
        padding=True,
        return_tensors="pt"
    )

    # Show that it can be saved and loaded
    print("\nLoading saved transformers tokenizer:")
    loaded_tokenizer = load_transformers_tokenizer("transformers_tokenizer_demo")
    test_encode = loaded_tokenizer(
        "Testing that loading works correctly.",
        return_tensors="pt"
    )
    print(f"Encoded IDs: {test_encode['input_ids'].tolist()[0]}")
    print(f"Decoded: {loaded_tokenizer.decode(test_encode['input_ids'][0])}")

    return transformers_tokenizer

In [ ]:
# Check if this even works...
demonstrate_workflow2()

## Create and train the tokenizer

In [ ]:
# Train a tokenizer if you don't have one yet
trained_tokenizer = create_gpt2_tokenizer(vocab_size=8192, min_frequency=2)

# This saves the tokenizer out to memory as well
trained_tokenizer = train_tokenizer_from_texts(tokenizer=trained_tokenizer, texts=data['train']['sentence'], batch_size=512, output_dir=models_dir)


In [ ]:
# This pulls the trained_tokenizer and makes it compatable with the transformers library for training
# Needs to be a PreTrainedTokenizerFast object
tokenizer_fast = convert_to_transformers_tokenizer(tokenizer_path=f"{models_dir}/gpt2_tokenizer.json",
                                                   output_dir=models_dir, model_max_length=256)

# Check the tokenizer
tokenizer_fast

In [ ]:
# Check to make sure the fast tokenizer is working
encodings = use_transformers_tokenizer(tokenizer_fast, data['validation']['sentence'][1], padding=True, max_length=1024, return_tensors='pt')
encodings

# Tokenize all of the data

## Functions

In [ ]:
def tokenize_dataset_dict(dataset_dict, tokenizer, max_length=None, batch_size=1000):
    """
    Tokenize all sentence entries in a Hugging Face DatasetDict.

    Args:
        dataset_dict: A Hugging Face DatasetDict containing data with 'sentence' column
        tokenizer: A transformers tokenizer
        max_length: Maximum sequence length for tokenization (None = no limit)
        batch_size: Batch size for processing

    Returns:
        A new DatasetDict with tokenized data
    """
    from datasets import DatasetDict

    # Function to tokenize a batch of examples
    def tokenize_function(examples):
        # For transformers tokenizers use the __call__ method
        return tokenizer(
            examples["sentence"],
            padding="max_length" if max_length else False,
            truncation=True if max_length else False,
            max_length=max_length,
            return_special_tokens_mask=True,
        )

    # Create a new DatasetDict to store the tokenized datasets
    tokenized_datasets = DatasetDict()

    # Process each split in the dataset
    for split_name, dataset in dataset_dict.items():
        # Map the tokenize function over the dataset in batches
        tokenized_datasets[split_name] = dataset.map(
            tokenize_function,
            batched=True,
            batch_size=batch_size,
            # remove_columns=["sentence"] if "sentence" in dataset.column_names else None, # Keep the sentences
            desc=f"Tokenizing {split_name} split",
        )

        # Print the first example to verify
        if len(tokenized_datasets[split_name]) > 0:
            print(f"\nExample from {split_name} split:")
            example = tokenized_datasets[split_name][0]
            print(f"Input IDs (first 10): {example['input_ids'][:10]}...")
            print(f"Attention Mask (first 10): {example['attention_mask'][:10]}...")

            # Decode a sample for verification
            decoded = tokenizer.decode(example['input_ids'])
            print(f"Decoded sample preview: {decoded[:50]}...")

    return tokenized_datasets

## Tokenize the Datasets

Do we need to do this?  I think so... I can train when I do this so... yes?

In [ ]:
# Tokenize the DatasetDict
# TODO: We can probably decrease the max_length here to 128 or 64 to save memory; We'll have to test it out
# Or maybe we don't use a max length...?? TODO: Figure out how to handle this
tokenize_dataset_dict = tokenize_dataset_dict(data, tokenizer_fast, max_length=256, batch_size=1000)
tokenize_dataset_dict

In [ ]:
# Make sure 'sentnece' is still there
tokenize_dataset_dict['validation']['sentence'][0]

In [ ]:
tokenize_dataset_dict['validation']['input_ids'][0]

In [ ]:
# A short function to easily convert from ids_to_tokens
# It effectivly just wraps the decode function
def ids_to_tokens(tokenizer, ids):
    '''
    Convert token IDs back to tokens using the tokenizer
    Really just a mapping function for tokenizer.decode()
    '''
    tokens = tokenizer.decode(ids)

    return tokens

In [ ]:
trained_tokenizer.decode(tokenize_dataset_dict['validation']['input_ids'][0])

In [ ]:
ids_to_tokens(trained_tokenizer, tokenize_dataset_dict['validation']['input_ids'][0])

In [ ]:
tokenize_dataset_dict['validation']['attention_mask'][0]

## Set formating to torch of the DatasetDict

In [ ]:
# Make sure the tokenized dataset is in the correct format for PyTorch

# old
# tokenize_dataset_dict.set_format(type='torch', columns=['sentence', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'])

# New
tokenize_dataset_dict.set_format(type='torch', columns=['sentence', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'])

### Check shapes

In [ ]:
# Note that these have been padded out to the max_length

for i in range(0, 11):
    print(tokenize_dataset_dict['train'][i]['input_ids'].shape)

In [ ]:
for i in range(0, 11):
    print(tokenize_dataset_dict['train'][i]['attention_mask'].shape)

In [ ]:
for i in range(0, 11):
    print(tokenize_dataset_dict['train'][i]['sentence'])

# Build the Decoder

## A Short Rebuild of the above but with Hugging Face API

We might be able to leverage accelerated training by using HF so this could be worth to have.  It also uses a PyTorch backend which I know works on Brandon's system... TensorFlow has been giving me problems in my environment lately.

In [ ]:
# Grab the decoder-only setup
from transformers import GPT2Config, GPT2LMHeadModel

In [ ]:
# Make the model

VOCAB_SIZE = 8912 # Define this based on training data vocab size / tokenize size.
# TODO: Or does it need to relate to the tokenizer?
SEQUENCE_LENGTH = 500 # Define this based on training data sequence length. aka. context window
EMBEDDING_DIM = 512 # Configure
FEED_FORWARD_DIM = 4 * EMBEDDING_DIM # Standard for decoder-only items.
DROPOUT = 0.5 # Configure
NUM_HEADS = 16 # Configure attention heads
NUM_DECODER_LAYERS = 4 # Configure

# Define the model configuration
model_config = GPT2Config(
    vocab_size=VOCAB_SIZE,
    n_positions=SEQUENCE_LENGTH,
    n_ctx=SEQUENCE_LENGTH,
    n_embd=EMBEDDING_DIM,
    n_layer=NUM_DECODER_LAYERS,
    n_head=NUM_HEADS,
    resid_pdrop=DROPOUT,
    embd_pdrop=DROPOUT,
    attn_pdrop=DROPOUT,
    # layer_norm_epsilon=1e-5,
    # initializer_range=0.02,
)

# Initialize the model
gptmodel = GPT2LMHeadModel(model_config)
# Output the model layers for inspection
gptmodel

In [ ]:
# Check the number of trainable parameters.
# How many trainable parameters exist in the gptmodel? (setting p.requires_grad to True for this)
trainable_params = sum(p.numel() for p in gptmodel.parameters() if p.requires_grad)
print("Number of trainable parameters:", trainable_params)

In [ ]:
# Print each parameter's name and trainable count individually
for name, param in gptmodel.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.numel()} trainable parameters")

There is a difference in the reported number of trained parameters between Etienne's custom build and the Hugging Face model.

I'm 90% certain it has to do with the positional eoncodings being done slightly different.  48M of his model parameters are from the PosEnc which doesn't show up the same way in the GPT-model.

Both models are likely to generate similar results.

# Training the Model

## Setup Training Arguments

In [ ]:
# Set up the Training Args
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5, # Prior runs show that I am easily over fitting the data at 25 epochs...
    weight_decay=0.01,
    logging_dir='./logs',
    # logging_steps=10, # This made my loss vs epoch plot too noisy...
    logging_strategy='epoch',
    save_strategy='epoch',
)

In [ ]:
# Set up the Trainer

#mlm is the masked language modeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer_fast, mlm=False, return_tensors='pt')

trainer = Trainer(
    model=gptmodel,
    args=training_args,
    train_dataset=tokenize_dataset_dict['train'],
    eval_dataset=tokenize_dataset_dict['validation'],
    processing_class=tokenizer_fast,
    data_collator=data_collator
)

## CHOO CHOO!

In [ ]:
# Run the Trainer
trainer.train()

In [ ]:
# Save out the model
trainer.save_model("../models/hf_gpt2_style")

# Metrics

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

We'll likely need to create our prompt input type thing down here...